In [1]:
import xbow
from xbow import instances
import boto3
ec2_resource = boto3.resource('ec2', region_name='eu-west-1')

In [2]:
insts = ec2_resource.instances.filter(Filters=[{'Name': 'key-name', 'Values': ['MyDaskScheduler']}])
print list(insts)
for instance in insts:
    print instance.key_name, instance.launch_time

[ec2.Instance(id='i-09f00a71e81246141')]
MyDaskScheduler 2018-02-05 17:48:00+00:00


In [3]:
inst = list(insts)[0]
ci = instances.ConnectedInstance(inst)

In [4]:
ci.exec_command('ls')
print(ci.output)

amber16
build_gromacs_2016.4.sh
build_gromacs.sh
cocojob.sh
dicker
install_cuda.sh
packages
popylar
scheduler.log
shared
TGTC
TGTC_GMX
tios



In [7]:
ci.upload('penta_w.crd', 'shared/CoCo-MD_example2/penta_w.crd')

In [8]:
ci.upload('penta_w.top', 'shared/CoCo-MD_example2/penta_w.top')
ci.upload('penta_w.pdb', 'shared/CoCo-MD_example2/penta_w.pdb')
ci.upload('min1.in', 'shared/CoCo-MD_example2/min1.in')
ci.upload('min2.in', 'shared/CoCo-MD_example2/min2.in')
ci.upload('md.in', 'shared/CoCo-MD_example2/md.in')

In [10]:
ci.upload('dryrun_cocomd_2.py', 'shared/CoCo-MD_example2/dryrun_cocomd_2.py')

In [11]:
ci.exec_command('sudo pip install xbowflow==0.0.4')
print(ci.output)

Downloading/unpacking xbowflow==0.0.4
  Could not find a version that satisfies the requirement xbowflow==0.0.4 (from versions: 0.0.1, 0.0.2, 0.0.3)
Cleaning up...
No distributions matching the version for xbowflow==0.0.4
Storing debug log for failure in /home/ubuntu/.pip/pip.log



In [6]:
insts = ec2_resource.instances.filter(Filters=[{'Name': 'key-name', 'Values': ['MyDaskWorkers']}])
print list(insts)
for instance in insts:
    print instance.key_name, instance.launch_time

[ec2.Instance(id='i-01ea2335c11872757'), ec2.Instance(id='i-00a2b27ad361a28b1')]
MyDaskWorkers

AttributeError: 'ec2.Instance' object has no attribute 'lau'

In [7]:
from xbow import pools
sip = pools.SpotInstancePool('MyDaskWorkers')
print sip.status

 ready


In [8]:
sip.exec_command('sudo pip install xbowflow==0.0.4')
print sip.outputs[0]

Downloading/unpacking xbowflow==0.0.3
  Running setup.py (path:/tmp/pip_build_root/xbowflow/setup.py) egg_info for package xbowflow
    * Checking Python version...
    * Python version OK!
    
  Running setup.py install for xbowflow
    * Checking Python version...
    * Python version OK!
    
Cleaning up...



In [14]:
sip.exec_command('/home/ubuntu/amber16/bin/pmemd.cuda')
print sip.outputs[0]


  Unit    5 Error on OPEN: mdin                                                                                                                                                                                                                                                            

  Unit    5 Error on OPEN: mdin                                                                                                                                                                                                                                                            
STOP PMEMD Terminated Abnormally!

